#  Notebook Statistika 3


In [1]:
# print("Hello, World!")

In [2]:
import pandas as pd
df = pd.DataFrame({'size': ['Small', 'Medium', 'Large', 'ExtraLarge']})
size_mapping = {'Small': 1, 'Medium': 2, 'Large': 3, 'ExtraLarge': 4}
df['size_encoded'] = df['size'].map(size_mapping)
df


,size,size_encoded
0,Small,1
1,Medium,2
2,Large,3
3,ExtraLarge,4


In [3]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
df = pd.DataFrame({'size': ['Small', 'Medium', 'Large', 'ExtraLarge']})
enc = OrdinalEncoder(categories='auto', dtype=int)
df['size_encoded'] =enc.fit_transform(df[['size']])
df

,size,size_encoded
0,Small,3
1,Medium,2
2,Large,1
3,ExtraLarge,0


In [4]:
# import category_encoders as ce
# import pandas as pd
# df = pd.DataFrame({'size': ['Small', 'Medium', 'Large', 'ExtraLarge']})
# encoder = ce.TargetEncoder(cols=['size'])
# df['size_encoded'] = encoder.fit_transform(df['size'],df['target'])
# df = pd.DataFrame({'size': ['Small', 'Medium', 'Large', 'ExtraLarge']})


# Statistika – 3 paskaita: Užduotys 

Dataset'ai: `tips` ir `penguins` (`seaborn.load_dataset`)

## A. Tikimybės (tips)
1. Apskaičiuokite P(tip > 5).
2. Apskaičiuokite P(smoker == 'Yes'). Patikrinkite nepriklausomumą: ar P(tip > 5 ∩ smoker) ≈ P(tip > 5) * P(smoker)?
3. Apskaičiuokite sąlyginę tikimybę P(tip > 5 | day ∈ {Sat, Sun}). Palyginkite su P(tip > 5 | day ∈ {Thur, Fri}).
4. Pavaizduokite `total_bill` histogramą (20 binų) ir pažymėkite vertikalią liniją ties 30.

## B. Kodavimas (tips)
5. Sukurkite one-hot/dummy kintamuosius stulpeliams: `sex`, `smoker`, `day`, `time`. Paaiškinkite, kada naudotumėte `drop_first=True`.
6. Susikurkite `tip_rate = tip / total_bill`. Pagal kvantiles sukurkite ordinal kintamąjį `service_level`: Low/Medium/High. Priskirkite 1/2/3.
7. Pavaizduokite `tip_rate` dėžės diagramą (boxplot) pagal `smoker`. Ką pastebite?



In [5]:
# 📦 Importai ir nustatymai
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sns  # tik duomenų įkėlimui
pd.set_option("display.max_columns", 100)


In [6]:
# 🔽 Įkeliame duomenų rinkinius
tips = sns.load_dataset('tips')
penguins = sns.load_dataset('penguins')

print("tips shape:", tips.shape)
display(tips.head())

print("\npenguins shape:", penguins.shape)
display(penguins.head())

tips shape: (244, 7)


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4



penguins shape: (344, 7)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [11]:
#  1) P(tip > 5).
p_tip5 = (tips['tip'] > 5).mean()
print("P(tip > 5):", round(p_tip5, 3))

P(tip > 5): 0.074


In [14]:
tips[tips['tip'] >5].shape[0] / tips.shape[0]

0.07377049180327869

In [13]:
tips.shape

(244, 7)

In [17]:
# 2) P('smoker'==yes)
p_smoker = (tips['smoker'] == 'Yes').mean()
p_both = ((tips['tip'] > 5) & (tips['smoker'] == 'Yes')).mean()
print("P('smoker' == 'Yes'):", round(p_smoker, 3))
print("P(tip>5 ∩ smoker) =", round( p_both, 3),
      "vs P(tip>5)*P(smoker) =", round((p_tip5 * p_smoker), 3))


P('smoker' == 'Yes'): 0.381
P(tip>5 ∩ smoker) = 0.025 vs P(tip>5)*P(smoker) = 0.028


In [18]:
# 3) P(tip>5 | weekend) vs (| workdays)
weekend = tips['day'].isin(['Sat', 'Sun'])
p_tip_weekend = tips.loc[weekend, 'tip'].gt(5).mean()
p_tip_work = tips.loc[~weekend, 'tip'].gt(5).mean()
print("P(tip>5 | weekend) =", round(p_tip_weekend, 3))
print("P(tip>5 | workdays) =", round(p_tip_work, 3))

P(tip>5 | weekend) = 0.092
P(tip>5 | workdays) = 0.037


In [19]:
# 4) Dvimatė dažnių lentelė ir P(smoker='Yes' | day='Sun')
ct = pd.crosstab(tips['smoker'], tips['day'])
display(ct)
p_smoker_yes_given_sun = ct.loc['Yes','Sun'] / ct['Sun'].sum()
print("P(smoker='Yes' | day='Sun') =", round(p_smoker_yes_given_sun, 3))

day,Thur,Fri,Sat,Sun
smoker,,,,
Yes,17,15,42,19
No,45,4,45,57


P(smoker='Yes' | day='Sun') = 0.25


In [26]:
# 5) One-hot/Dummies
cat_cols = ['sex','smoker','day','time']
onehot = pd.get_dummies(tips, columns=cat_cols, drop_first=False)

onehot.shape
onehot.head()

# dummies_drop = pd.get_dummies(tips, columns=cat_cols, drop_first=True)
# onehot.shape, dummies_drop.shape

,total_bill,tip,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,1.01,2,False,True,False,True,False,False,False,True,False,True
1,10.34,1.66,3,True,False,False,True,False,False,False,True,False,True
2,21.01,3.50,3,True,False,False,True,False,False,False,True,False,True
3,23.68,3.31,2,True,False,False,True,False,False,False,True,False,True
4,24.59,3.61,4,False,True,False,True,False,False,False,True,False,True


In [25]:
tips['day'].groupby(tips['day']).count()

C:\Users\krist\AppData\Local\Temp\ipykernel_23796\2920168207.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips['day'].groupby(tips['day']).count()


day
Thur    62
Fri     19
Sat     87
Sun     76
Name: day, dtype: int64